Load the required modules

In [236]:
import pandas as pd
import numpy as np

In [237]:
# Foursquare Credentials
CLIENT_ID = 'A1IAGYAM3K4MZV2UEBJZ4BFHDYBJZP2OJTGAUW25LX5QNUG1' # your Foursquare ID
CLIENT_SECRET = 'HIRPIFJKO5ZQQUAJ5FC0HAXVPKAZBIFH1B0J3MZXPIRLGWUZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 400 # Limit results to 400

In [238]:
# Define custom functions

from math import sin,cos,radians,asin,sqrt # the sine, cosine, radians, arcsin, square root functions
import requests

def distance(coor1, coor2):   
    """This function calculates the distance (in kilometres) between two sets of latitude/longitude coordinates (coor1 and coor2), using the haversine formula (https://en.wikipedia.org/wiki/Haversine_formula)"""
    hav = lambda x: sin(x/2)**2
    coor1[0], coor1[1], coor2[0], coor2[1] = radians(coor1[0]), radians(coor1[1]), radians(coor2[0]), radians(coor2[1])
    return(2*6371*asin(sqrt(hav(coor2[0] - coor1[0]) + cos(coor1[0])*cos(coor2[0])*hav(coor2[1] - coor1[1]))))


def getNearbyVenues(names, latitudes, longitudes, query="restaurant", radius=3000):
    """This function returns the nearby venues at the specified location from the Foursquare API within 3km"""
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
            
        # make the GET request
        for a in range(0,5):
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']

                # return only relevant information for each nearby venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
                break
            except:
                print(str(name) + " - error in accessing venues. Making another attempt...")

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    nearby_venues.drop_duplicates(subset = ["Venue Latitude", "Venue Longitude", "Venue"], inplace = True)
    nearby_venues.reset_index(drop = True, inplace = True)
    
    return(nearby_venues)


In [241]:
# Load the Australian postcodes dataset and prepare the data.

postcodes = pd.read_csv('AusPostcode.csv')

# Clean up the postcodes. Each postcode should only be represented once.
postcodes = postcodes.groupby(["Postcode", "State", "Latitude", "Longitude"], sort = False, as_index = False).agg('|'.join)

# Drop all of the postcodes that are not located in Victoria (the state where Melbourne is located).
# All of the Victorian postcodes range from 3000 to 3999.
postcodes.drop(postcodes[postcodes["Postcode"]<3000].index, inplace = True)
postcodes.drop(postcodes[postcodes["Postcode"]>3999].index, inplace = True)
postcodes.reset_index(drop=True, inplace = True)

# Obtain latitude and longitude for the city centre (postcode 3000)
CC_lat = postcodes[postcodes["Postcode"]==3000]["Latitude"].values[0]
CC_lon = postcodes[postcodes["Postcode"]==3000]["Longitude"].values[0]

# Calculate the distance from the city centre for each postcode
DistanceKM = []
for a in range(0,postcodes.shape[0]):
    lat = postcodes.loc[a, "Latitude"]
    lon = postcodes.loc[a, "Longitude"]
    DistanceKM.append(distance([CC_lat, CC_lon], [lat,lon]))
postcodes["FromCity_KM"] = DistanceKM

# Drop all of the postcodes that are more than 60 kilometres from the city centre
postcodes.drop(postcodes[postcodes["FromCity_KM"]>60].index, inplace = True)
postcodes.reset_index(drop=True, inplace = True)

postcodes.shape

(269, 6)

In [243]:
# Download list of restaurants from the Foursquare API
melbourne_venues = getNearbyVenues(names=postcodes['Postcode'],
                                   latitudes=postcodes['Latitude'],
                                   longitudes=postcodes['Longitude'],
                                   query="Restaurant"
                                  )

melbourne_venues.shape

3000
3002
3003
3004
3005
3006
3008
3010
3011
3012
3013
3015
3016
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3070
3071
3072
3073
3074
3075
3076
3078
3078 - error in accessing venues. Making another attempt...
3079
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3093
3094
3095
3096
3097
3099
3101
3102
3103
3104
3105
3106
3107
3108
3109
3113
3114
3115
3116
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3204
3205
3206
3207
3211
3212
3220


(4251, 7)

In [198]:
# Get specifically Persian restaurants
persian_venues = getNearbyVenues(names=postcodes['Postcode'],
                                   latitudes=postcodes['Latitude'],
                                   longitudes=postcodes['Longitude'],
                                   query="Persian restaurant"
                                  )

persian_venues

3000
3002
3003
3004
3005
3006
3008
3010
3011
3012
3013
3015
3016
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3070
3071
3072
3073
3074
3075
3076
3078
3079
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3093
3094
3095
3096
3097
3099
3101
3102
3103
3104
3105
3106
3107
3108
3109
3113
3114
3115
3116
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3204
3205
3206
3207
3211
3212
3220
3222
3223
3335
3337
3338
3340
3353
3427
3428
3429
3430
3431


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,3000,-37.81,144.97,Arabesque,-37.803363,144.958384,Persian Restaurant
1,3000,-37.81,144.97,Persiangrill,-37.817192,144.964624,Persian Restaurant
2,3000,-37.81,144.97,Fars Food,-37.817608,144.958000,Persian Restaurant
3,3002,-37.82,144.99,Hafez,-37.819495,145.006808,Persian Restaurant
4,3002,-37.82,144.99,HAFEZ Persian Restaurant,-37.819499,145.006939,Persian Restaurant
5,3012,-37.81,144.85,Salamatea,-37.786648,144.833020,Persian Restaurant
6,3044,-37.73,144.94,habib,-37.743507,144.966182,Persian Restaurant
7,3072,-37.74,145.03,Padideh Kebab,-37.724994,145.027854,Persian Restaurant
8,3101,-37.81,145.04,AliQapu,-37.793986,145.063139,Persian Restaurant
9,3102,-37.80,145.05,Persian House,-37.803300,145.082950,Persian Restaurant


In [244]:
# For each of the suburbs, calculate the distance to the nearest Persian restaurant

def distance_from_closest(lat, long, venues):
    """From a nominated latitude (lat) and longitude (lon), this function will return the distance of the closest point from a set of coordinates (venues)"""
    dist = []
    for a in range(0, venues.shape[0]):
        dist.append(distance([lat,long], [venues.loc[a, "Venue Latitude"], venues.loc[a, "Venue Longitude"]]))
    return(np.min(dist))

DistToNearestPersian = []

for b in range(0, postcodes.shape[0]):
    DistToNearestPersian.append(distance_from_closest(postcodes.loc[b, "Latitude"],
                                                      postcodes.loc[b, "Longitude"],
                                                      persian_venues))

postcodes["FromNearestPersian_KM"] = DistToNearestPersian
postcodes


,Postcode,State,Latitude,Longitude,Suburb,FromCity_KM,FromNearestPersian_KM
0,3000,VIC,-37.81,144.97,Melbourne,0.000000,0.928745
1,3002,VIC,-37.82,144.99,East Melbourne,2.079186,1.477444
2,3003,VIC,-37.81,144.94,West Melbourne,2.635480,1.775732
3,3004,VIC,-37.84,144.98,Melbourne|St Kilda Road Central,3.449539,2.873263
4,3005,VIC,-37.82,144.95,World Trade Centre,2.079186,0.751363
...,...,...,...,...,...,...,...
264,3975,VIC,-38.05,145.26,Lynbrook|Lyndhurst,36.866145,8.263263
265,3976,VIC,-38.03,145.27,Hampton Park,35.929579,6.997995
266,3977,VIC,-38.21,145.32,Botanic Ridge|Cannons Creek|Cranbourne|Cranbou...,54.023573,26.670345
267,3978,VIC,-38.15,145.42,Cardinia|Clyde|Clyde North,54.634163,25.716198


In [256]:
# For each of the suburbs, calculate the total number of restaurants within a 3km radius
# Note that there is some overlap; a venue may be counted more than once for adjacent suburbs

n_venues = []
for b in range(0, postcodes.shape[0]):
    lat1 = postcodes.loc[b, "Latitude"]
    lon1 = postcodes.loc[b, "Longitude"]    
    dist = []
    for c in range(0, melbourne_venues.shape[0]):
        lat2 = melbourne_venues.loc[c, "Venue Latitude"]
        lon2 = melbourne_venues.loc[c, "Venue Longitude"]
        dist.append(distance([lat1, lon1], [lat2, lon2]))
    melbourne_venues["FromPostcode_KM"] = dist
    n_venues.append(melbourne_venues[melbourne_venues["FromPostcode_KM"]<=3].shape[0])

postcodes["NumVenues"] = n_venues
postcodes.head()

,Postcode,State,Latitude,Longitude,Suburb,FromCity_KM,FromNearestPersian_KM,NumVenues
0,3000,VIC,-37.81,144.97,Melbourne,0.000000,0.928745,347
1,3002,VIC,-37.82,144.99,East Melbourne,2.079186,1.477444,340
2,3003,VIC,-37.81,144.94,West Melbourne,2.635480,1.775732,241
3,3004,VIC,-37.84,144.98,Melbourne|St Kilda Road Central,3.449539,2.873263,230
4,3005,VIC,-37.82,144.95,World Trade Centre,2.079186,0.751363,257
5,3006,VIC,-37.82,144.97,South Wharf|Southbank,1.111949,0.566114,383
6,3008,VIC,-37.81,144.95,Docklands,1.756987,1.042671,291
7,3010,VIC,-37.80,144.96,University Of Melbourne,1.417140,0.400017,320
8,3011,VIC,-37.80,144.90,Footscray|Seddon|Seddon West,6.249586,5.143218,137
9,3012,VIC,-37.81,144.85,Brooklyn|Kingsville|Kingsville West|Maidstone|...,10.541919,2.994708,51


In [278]:
# Calculate the diversity of each neighbourhood, in terms of the variety of restaurant type within 3km of its coordinates

def gini_simpson(cat_list):
    """Calculates the Gini-Simpson diversity index from a list of categories (cat_list) provided.
    See: https://en.wikipedia.org/wiki/Diversity_index#Gini%E2%80%93Simpson_index
    Higher values indicate greater diversity."""
    unq_list = np.unique(np.array(cat_list), return_counts = True)
    p = unq_list[1] / np.sum(unq_list[1])
    return(1 - np.sum(p**2))

diversity = []
for b in range(0, postcodes.shape[0]):
    lat1 = postcodes.loc[b, "Latitude"]
    lon1 = postcodes.loc[b, "Longitude"]    
    dist = []
    for c in range(0, melbourne_venues.shape[0]):
        lat2 = melbourne_venues.loc[c, "Venue Latitude"]
        lon2 = melbourne_venues.loc[c, "Venue Longitude"]
        dist.append(distance([lat1, lon1], [lat2, lon2]))
    melbourne_venues["FromPostcode_KM"] = dist
    diversity.append(gini_simpson(melbourne_venues.loc[melbourne_venues["FromPostcode_KM"]<=3, "Venue Category"].values))

postcodes["Diversity"] = diversity

# If there are fewer than 10 venues, don't calculate a diversity score
postcodes.loc[postcodes["NumVenues"]<10, "Diversity"] = np.nan

,Postcode,State,Latitude,Longitude,Suburb,FromCity_KM,FromNearestPersian_KM,NumVenues,Diversity
100,3115,VIC,-37.74,145.27,Wonga Park,27.492147,14.305841,1,0.000000
101,3116,VIC,-37.75,145.33,Chirnside Park,32.334368,19.272768,21,0.879819
102,3121,VIC,-37.83,145.01,Burnley|Burnley North|Cremorne|Richmond|Richmo...,4.158171,1.198210,195,0.882472
103,3122,VIC,-37.84,145.05,Auburn South|Glenferrie South|Hawthorn|Hawthor...,7.778164,4.217394,159,0.853685
104,3123,VIC,-37.83,145.04,Auburn|Hawthorn East,6.538444,3.129779,162,0.847432
105,3124,VIC,-37.82,145.06,Camberwell|Camberwell North|Camberwell South|C...,7.983718,2.740973,180,0.826728
106,3125,VIC,-37.84,145.12,Bennettswood|Burwood|Surrey Hills South,13.590480,2.337380,125,0.922368
107,3126,VIC,-37.84,145.09,Camberwell East|Canterbury,11.055079,1.294857,99,0.828079
108,3127,VIC,-37.82,145.10,Mont Albert|Surrey Hills|Surrey Hills North,11.473647,2.385724,115,0.881210
109,3128,VIC,-37.82,145.12,Box Hill|Box Hill Central|Box Hill South|Houst...,13.223341,3.747219,110,0.922645


In [317]:
# Drop suburbs that are more than 30 kilometres from the city.

postcodes_filtered = postcodes.copy()
postcodes_filtered.drop(postcodes_filtered[postcodes_filtered["FromCity_KM"]>30].index, inplace = True)
postcodes_filtered.reset_index(drop=True, inplace = True)

In [315]:
# Rank each suburb
postcodes_filtered["FromNearestPersian_KM_rank"] = postcodes_filtered["FromNearestPersian_KM"].rank(ascending = False)
postcodes_filtered["NumVenues_rank"] = postcodes_filtered["NumVenues"].rank(ascending = False)
postcodes_filtered["Diversity_rank"] = postcodes_filtered["Diversity"].rank(ascending = False)

overall = []
for a in range(0,postcodes_filtered.shape[0]):  
    overall.append(np.sum([postcodes_filtered.loc[a, "FromNearestPersian_KM_rank"],
                           postcodes_filtered.loc[a, "NumVenues_rank"],
                           postcodes_filtered.loc[a, "Diversity_rank"]]))

postcodes_filtered["Overall"] = overall
postcodes_filtered.sort_values(by=["Overall"])[0:21]

,Postcode,State,Latitude,Longitude,Suburb,FromCity_KM,FromNearestPersian_KM,NumVenues,Diversity,FromNearestPersian_KM_rank,NumVenues_rank,Diversity_rank,Overall
112,3135,VIC,-37.82,145.25,Bedford Road|Heathmont|Ringwood East,24.621260,13.153762,65,0.926391,21.0,96.0,10.0,127.0
123,3150,VIC,-37.88,145.17,Brandon Park|Glen Waverley|Wheelers Hill,19.209175,7.207298,113,0.943848,69.0,60.0,1.0,130.0
111,3134,VIC,-37.81,145.23,Heathwood|Ringwood|Ringwood North|Warrandyte S...,22.840819,11.104256,61,0.930395,30.0,104.0,8.0,142.0
108,3131,VIC,-37.84,145.18,Brentford Square|Forest Hill|Nunawading,18.743837,7.387722,102,0.924260,65.0,65.5,12.0,142.5
110,3133,VIC,-37.84,145.19,Vermont|Vermont South,19.608752,8.255991,93,0.924962,59.0,75.5,11.0,145.5
141,3179,VIC,-37.86,145.26,Scoresby,26.067466,14.357311,63,0.915092,15.0,101.0,33.0,149.0
133,3168,VIC,-37.93,145.12,Clayton|Notting Hill,18.745865,9.072770,79,0.923249,50.0,84.5,15.0,149.5
134,3169,VIC,-37.94,145.10,Clarinda|Clayton South,18.416103,9.947160,52,0.943047,43.0,111.0,2.0,156.0
132,3167,VIC,-37.93,145.10,Oakleigh South,17.557328,8.835946,62,0.940687,53.0,103.0,3.0,159.0
154,3192,VIC,-37.96,145.06,Cheltenham|Cheltenham East|Cheltenham North|So...,18.454855,12.571164,88,0.902118,25.0,78.5,57.0,160.5


In [313]:
# Create a map of the top 20 recommended locations

import folium

latitude = melbourne_venues[melbourne_venues["Neighborhood"]==3000]["Neighborhood Latitude"].values[0]
longitude = melbourne_venues[melbourne_venues["Neighborhood"]==3000]["Neighborhood Longitude"].values[0]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []

for lat, lon, suburb in zip(postcodes_filtered.sort_values(by=["Overall"])[0:21]["Latitude"].values,
                            postcodes_filtered.sort_values(by=["Overall"])[0:21]["Longitude"].values,
                            postcodes_filtered.sort_values(by=["Overall"])[0:21]["Suburb"].values):
    label = folium.Popup(suburb, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)

map_clusters